In [0]:

file_location = "/FileStore/tables/trips.csv"
file_type = "csv"

infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.write.format("delta") \
.save("/delta/bronze_trips")

In [0]:

df = spark.read.format("delta") \
    .load("/delta/bronze_trips")

df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_trips")

In [0]:
%sql
--enabling property to update column name
ALTER TABLE silver_trips SET TBLPROPERTIES (
    'delta.minReaderVersion' = '2',
    'delta.minWriterVersion' = '5',
    'delta.columnMapping.mode' = 'name'
  )

In [0]:
%sql
--renaming default column names to reflect more descriptive, column names
ALTER TABLE silver_trips 
RENAME COLUMN _c0 TO trip_id;

ALTER TABLE silver_trips 
RENAME COLUMN _c1 TO rideable_type;

ALTER TABLE silver_trips 
RENAME COLUMN _c2 TO start_at;

ALTER TABLE silver_trips 
RENAME COLUMN _c3 TO ended_at;

ALTER TABLE silver_trips 
RENAME COLUMN _c4 TO start_station_id;

ALTER TABLE silver_trips 
RENAME COLUMN _c5 TO end_station_id;

ALTER TABLE silver_trips 
RENAME COLUMN _c6 TO rider_id;


com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Missing field _c0 in table spark_catalog.default.silver_trips with schema:
root
-- trip_id: string (nullable = true)
-- rideable_type: string (nullable = true)
-- start_at: string (nullable = true)
-- ended_at: string (nullable = true)
-- start_station_id: string (nullable = true)
-- end_station_id: string (nullable = true)
-- rider_id: string (nullable = true)
.; line 2 pos 0
	at org.apache.spark.sql.errors.QueryCompilationErrors$.missingFieldError(QueryCompilationErrors.scala:3419)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFieldNameAndPosition$.$anonfun$resolveFieldNames$1(Analyzer.scala:5208)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFieldNameAndPosition$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveFieldNameAndPosition$$resolveFieldNames(Analyzer.scala:5208)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFieldNameAndPosition$$anonfun$apply$62$$anonfun$applyOrElse$264.applyOrElse(Analyzer.scala:5121)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFieldNameAndPosition$$anonfun$apply$62$$anonfun$applyOrElse$264.applyOrElse(Analyzer.scala:5120)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:465)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:69)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:465)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$transformExpressionsDownWithPruning$1(QueryPlan.scala:171)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$1(QueryPlan.scala:212)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:69)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:212)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.recursiveTransform$1(QueryPlan.scala:223)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$4(QueryPlan.scala:233)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:308)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:233)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsDownWithPruning(QueryPlan.scala:171)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsWithPruning(QueryPlan.scala:142)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressions(QueryPlan.scala:120)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFieldNameAndPosition$$anonfun$apply$62.applyOrElse(Analyzer.scala:5120)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFieldNameAndPosition$$anonfun$apply$62.applyOrElse(Analyzer.scala:5103)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$3(AnalysisHelper.scala:141)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:69)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$1(AnalysisHelper.scala:141)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:372)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning(AnalysisHelper.scala:137)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning$(AnalysisHelper.scala:133)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUpWithPruning(LogicalPlan.scala:33)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUp(AnalysisHelper.scala:114)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUp$(AnalysisHelper.scala:113)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOpera

In [ ]:

payTbl = spark.table("silver_payments")
tripsTbl = spark.table("silver_trips")
stnTbl = spark.table("silver_stations")
stnTbl2 = spark.table("silver_stations")
ridersTbl = spark.table("silver_riders")

from pyspark.sql.functions import *
fact_trips_df = payTbl.join(ridersTbl, payTbl.rider_id == ridersTbl.rider_id,"inner") \
    .join(tripsTbl, ridersTbl.rider_id == tripsTbl.rider_id,"inner") \
        .join(stnTbl, tripsTbl.start_station_id == stnTbl.station_id,"inner") \
            .join(stnTbl2, tripsTbl.end_station_id == stnTbl2.station_id,"inner") \
    .withColumn("age_at_trip", months_between(col("start_at"),col("birth_date"))/lit(12)) \
        .withColumn("minutes_between", (to_timestamp(col("ended_at")) - to_timestamp(col("start_at"))).cast("bigInt")/60) \
    .select(tripsTbl["trip_id"].alias("trip_key"), payTbl["payment_id"].alias("payment_key"), payTbl["rider_id"].alias("rider_key"), stnTbl["station_id"].alias("station_key"), payTbl["date"].alias("date_key"), floor("age_at_trip").alias("age_at_trip"), format_number("minutes_between",2).alias("trip_duration_min_sec")).dropDuplicates().sort("trip_id")                
                
fact_trips_df.show()

fact_trips_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("fact_trips")